# 监督学习过程

## 1. 基础代码

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import re

from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

mnist = tf.keras.datasets.mnist
models = tf.keras.models
layers = tf.keras.layers
activations = tf.keras.activations
optimizers = tf.keras.optimizers
losses = tf.keras.losses

# 获取数据
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 处理数据
x_train, x_test = x_train / 255.0, x_test / 255.0

# 创建模型
model = models.Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation=activations.relu),
        layers.Dense(10, activation=activations.softmax)
    ])

# 编译模型
model.compile(
        optimizer=optimizers.Adam(),
        loss=losses.sparse_categorical_crossentropy,
        metrics=['accuracy']
)

# 显示模型
model.summary()

## 2. 定义一些帮助函数

In [ ]:
# 每次执行生成一个log文件夹
def __get_result_file_names():
    # 获取最大索引
    results_path = os.path.join(os.path.abspath('.'), 'results')
    index = 0
    for x in os.listdir(results_path):
        matched = re.match(r'^run-(\d{4})$', x)
        if matched:
            cur = int(matched.group(1))
            if cur > index:
                index = cur
    index += 1

    # 准备目录
    log_dir = './results/run-{0:0>4d}/logs'.format(index)
    model_file_name = './results/run-{0:0>4d}/model.h5'.format(index)
    return log_dir, model_file_name


def generate_callbacks_include_early_stop():
    log_dir, model_file_name = __get_result_file_names()
    
    return [
        # 保存Tensorboard日志
        TensorBoard(log_dir=log_dir),
        
        # 保存最优质模型
        ModelCheckpoint(
            filepath=model_file_name,
            monitor='val_loss',
            save_best_only=True
        ),
        
        # 提前结束训练，假如精度不再提升
        EarlyStopping(monitor='acc', patience=5),
        
        # 提前结束训练，假如验证损失不再改善
        EarlyStopping(monitor='val_loss', patience=5),
        
        # 根据指标调整学习率
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
    ]

## 2. 通过配置callbacks启用监督学习

启动tensorboard

```shell
$ docker exec -it tf-py3-jupyter /bin/sh -c 'tensorboard --bind_all --logdir /tf/results'
```

In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=10000,
    validation_split=0.2,
    callbacks=generate_callbacks_include_early_stop()
)